# Import libraries

In [1]:
from transformers import AutoModel, AutoTokenizer
from bertopic import BERTopic
from bertopic.backend import BaseEmbedder
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import KeyBERTInspired
import itertools
import numpy as np
import json
import os
from collections import defaultdict
import pandas as pd
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from underthesea import word_tokenize
from torch.utils.data import Dataset

c:\Users\rreip\anaconda3\envs\paddle_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

In [2]:
def extract_title_and_content(input_path):
    with open(input_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    title = ""
    content_lines = []
    in_content = False

    for line in lines:
        if line.startswith("Title:"):
            title = line.replace("Title:", "", 1).strip()
        elif line.startswith("Content:"):
            in_content = True
            continue  # bỏ dòng "Content:"
        elif in_content:
            content_lines.append(line.rstrip())

    content = ". " + "\n".join(content_lines)
    result = (title + "\n" + content).replace("\n", " ")
    
    return result


In [22]:
input_path = r"C:\Users\rreip\Downloads\HK6\IE403\Đồ án\before_bertopic_dataset\Cluster_001\original\1.txt"
output_path = extract_title_and_content(input_path)
output_path

'Các nước chia buồn vụ tai nạn máy bay của Ai Cập . Tổng thư ký Tổ chức Hiệp ước Bắc Đại Tây Dương (NATO) Jens Stoltenberg ngày 19/5 cho biết, nếu Ai Cập đề nghị, liên minh này sẽ hỗ trợ công tác tìm kiếm chiếc máy bay mang số hiệu MS 804 của hãng hàng không Ai Cập chở 66 người mất tích trước đó cùng ngày. “Tôi gửi lời chia buồn sâu sắc nhất đến những ai bị ảnh hưởng bởi vụ việc này. Tôi cũng gửi lời chia buồn sâu sắc đến Pháp và Ai Cập. Tôi biết rằng đã có những nỗ lực tìm kiếm cứu nạn ở mức độ quốc gia. Pháp và Ai Cập đang phối hợp trong công tác này cũng như việc điều tra. Chúng tôi sẽ tiếp tục theo dõi chặt chẽ diễn biến và nếu được đề nghị, NATO luôn sẵn sàng giúp đỡ”, ông Jens Stoltenberg nói. Thủ tướng Italy Matteo Renzi ngày 19/5 cũng đã gửi lời chia buồn, đồng thời bày tỏ sự đoàn kết với Ai Cập sau vụ máy bay của hãng hàng không Ai Cập mất tích trên Địa Trung Hải khi đang trên đường bay từ Paris đến Cairo. Trước đó, Hãng hàng không quốc gia Ai Cập (EgyptAir) xác nhận phía Hy L

In [27]:
base_dir = r"C:\Users\rreip\Downloads\HK6\IE403\Đồ án\before_bertopic_dataset"
all_clusters_docs = []

for i in range(1, 301):
    cluster_name = f"Cluster_{i:03d}"
    original_path = os.path.join(base_dir, cluster_name, "original")

    cluster_docs = []

    if os.path.exists(original_path):
        for filename in os.listdir(original_path):
            file_path = os.path.join(original_path, filename)
            if os.path.isfile(file_path):
                cleaned_text = extract_title_and_content(file_path)
                all_clusters_docs.append(cleaned_text)
    else:
        print(f"⚠️ Không tìm thấy thư mục: {original_path}")

# Ghi ra file tổng hợp
output_path = r"C:\Users\rreip\Downloads\HK6\IE403\Đồ án\all_events_dataset\all_events_dataset_khong_theo_event.txt"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(all_clusters_docs, f, ensure_ascii=False, indent=2)

In [17]:
with open(r"C:\Users\rreip\Downloads\HK6\IE403\Đồ án\all_events_dataset\all_events_dataset_khong_theo_event.txt", "r", encoding="utf-8") as f:
    list_of_docs = json.load(f)

print(len(list_of_docs))

1945


In [25]:
def convert_to_dataframe(data):
    rows = []
    for event_id, articles in enumerate(data):
        for article in articles:
            rows.append({
                "event_id": event_id + 1,
                "text": article
            })
    df = pd.DataFrame(rows)
    df["id"] = range(len(df))
    return df[["id", "event_id", "text"]]

with open(r"C:\Users\rreip\Downloads\HK6\IE403\Đồ án\all_events_dataset\all_events_dataset.txt", "r", encoding="utf-8") as f:
    data = json.load(f)
df = convert_to_dataframe(data)
print(df)

        id  event_id                                               text
0        0         1  Các nước chia buồn vụ tai nạn máy bay của Ai C...
1        1         1  Máy bay Ai Cập rơi: Những câu hỏi cho chính ph...
2        2         1  Giải mã bí ẩn máy bay rơi của EgyptAir . Phó C...
3        3         1  Phát hiện mảnh vỡ nghi của máy bay MS804 gặp n...
4        4         1  Máy bay EgyptAir có thể bị tấn công bằng tên l...
...    ...       ...                                                ...
1940  1940       300  Xử lý xong sự cố ở sân bay Buôn Ma Thuột . Chi...
1941  1941       300  Đã khắc phục xong sự cố đường băng sân bay Buô...
1942  1942       300  Khắc phục xong sự cố Cảng hàng không Buôn Ma T...
1943  1943       300  Khắc phục xong sự cố tại Cảng Hàng không Buôn ...
1944  1944       300  Sân bay Buôn Mê Thuột hoạt động trở lại . Trướ...

[1945 rows x 3 columns]


# Embedding

In [4]:
class ContrastivePairDataset(Dataset):
    def __init__(self, pairs, tokenizer, max_length=256):
        self.pairs = pairs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        text1, text2, label = self.pairs[idx]

        inputs1 = self.tokenizer(
            text1,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        inputs2 = self.tokenizer(
            text2,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        item = {
            'input_ids_1': inputs1['input_ids'].squeeze(0),
            'attention_mask_1': inputs1['attention_mask'].squeeze(0),
            'input_ids_2': inputs2['input_ids'].squeeze(0),
            'attention_mask_2': inputs2['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.float)
        }
        return item

In [5]:
def make_contrastive_pairs(df, max_neg_per_pos=1):
    from collections import defaultdict
    import random

    grouped = defaultdict(list)
    for _, row in df.iterrows():
        grouped[row['event_id']].append(row['text'])

    pairs = []
    event_ids = list(grouped.keys())
    for eid in event_ids:
        texts = grouped[eid]
        for i in range(len(texts)):
            for j in range(i + 1, len(texts)):
                pairs.append((texts[i], texts[j], 1.0))  # positive pair

                for _ in range(max_neg_per_pos):
                    neg_eid = random.choice([e for e in event_ids if e != eid])
                    neg_text = random.choice(grouped[neg_eid])
                    pairs.append((texts[i], neg_text, 0.0))  # negative pair

    random.shuffle(pairs)
    return pairs

In [6]:
def get_cls_embedding(model, input_ids, attention_mask):
    output = model(input_ids=input_ids, attention_mask=attention_mask)
    return output.last_hidden_state[:, 0]  # CLS token

In [7]:
from torch import nn
from torch.utils.data import DataLoader

def train_phobert_contrastive(model, tokenizer, pairs, device="cpu", epochs=1, batch_size=8, lr=2e-5):
    dataset = ContrastivePairDataset(pairs, tokenizer)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    loss_fn = nn.CosineEmbeddingLoss()

    model.train()
    model.to(device)

    for epoch in range(epochs):
        total_loss = 0
        for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}"):
            input_ids_1 = batch['input_ids_1'].to(device)
            attention_mask_1 = batch['attention_mask_1'].to(device)
            input_ids_2 = batch['input_ids_2'].to(device)
            attention_mask_2 = batch['attention_mask_2'].to(device)
            labels = batch['label'].to(device) * 2 - 1  # convert 0/1 to -1/+1

            emb1 = get_cls_embedding(model, input_ids_1, attention_mask_1)
            emb2 = get_cls_embedding(model, input_ids_2, attention_mask_2)

            loss = loss_fn(emb1, emb2, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")

    return model


In [8]:
# Khởi tạo model/tokenizer
model_name = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModel.from_pretrained(model_name)

# Chuẩn bị data
pairs = make_contrastive_pairs(df)

# Huấn luyện
trained_model = train_phobert_contrastive(model, tokenizer, pairs)
trained_model.save_pretrained("phobert_event_embedding")
tokenizer.save_pretrained("phobert_event_embedding")

Epoch 1: 100%|██████████| 1434/1434 [3:41:09<00:00,  9.25s/it]  


Epoch 1 Loss: 145.6047


('phobert_event_embedding\\tokenizer_config.json',
 'phobert_event_embedding\\special_tokens_map.json',
 'phobert_event_embedding\\vocab.txt',
 'phobert_event_embedding\\bpe.codes',
 'phobert_event_embedding\\added_tokens.json')

In [9]:
class PhoBERTEmbedder:
    def __init__(self, model_name="vinai/phobert-base", batch_size=32, max_length=256): 
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
        self.model = AutoModel.from_pretrained(model_name).to(self.device)
        self.batch_size = batch_size
        self.max_length = max_length

    def __call__(self, documents, show_progress_bar=False):
        self.model.eval()
        all_embeddings = []
        iterator = range(0, len(documents), self.batch_size)
        if show_progress_bar:
            iterator = tqdm(iterator, desc="Embedding documents")

        for i in iterator:
            batch_docs = documents[i:i+self.batch_size]
            tokens = self.tokenizer(
                batch_docs,
                padding=True,
                truncation=True,
                max_length=self.max_length,
                return_tensors="pt"
            ).to(self.device)

            with torch.no_grad():
                outputs = self.model(**tokens)
                cls_embeddings = outputs.last_hidden_state[:, 0, :]

            all_embeddings.append(cls_embeddings.cpu())

        return torch.cat(all_embeddings).numpy()

In [10]:
phobert_embedder = PhoBERTEmbedder(model_name="phobert_event_embedding")
embeddings = phobert_embedder(df['text'].tolist(), show_progress_bar=True)

Embedding documents: 100%|██████████| 61/61 [06:09<00:00,  6.06s/it]


# UMAP + HDBSCAN

In [11]:
# Grid tham số
umap_n_neighbors = [4, 7, 10]
umap_n_components = [5]
umap_min_dist = [0.0, 0.1] 
umap_metric = ["cosine"] #, "manhattan", "euclidean"

hdbscan_min_cluster_size = [4, 6, 8]
hdbscan_cluster_selection_method = ['eom', 'leaf'] 
hdbscan_metric = ['euclidean']

In [12]:
# Tổ hợp tham số
param_grid = list(itertools.product(
    umap_n_neighbors,
    umap_n_components,
    umap_min_dist,
    umap_metric,
    hdbscan_min_cluster_size,
    hdbscan_cluster_selection_method,
    hdbscan_metric
))

# Vectorizer

In [13]:
# Đọc stopword list từ file
with open(r"C:\Users\rreip\Downloads\HK6\IE403\Đồ án\vietnamese-stopwords.txt", "r", encoding="utf-8") as f:
    my_stopwords = [line.strip() for line in f if line.strip()]

In [14]:
vectorizer_model = CountVectorizer(stop_words=my_stopwords, ngram_range=(1, 2)) #max_df=0.9, 

# Representation

In [15]:
keybert_model = KeyBERTInspired()
representation_model = {
    "KeyBERT": keybert_model
}

# Training

In [31]:
# Tìm best_model
best_model = None
best_score = -np.inf
best_params = None

for i, (n_neighbors, n_components, min_dist, umap_metric, min_cluster_size, cluster_selection_method, hdbscan_metric) in enumerate(param_grid):
    print(f"\n🔍 Grid {i+1}/{len(param_grid)}: UMAP(n_neighbors={n_neighbors}, min_dist={min_dist}, metric={umap_metric}), "
        f"HDBSCAN(min_cluster_size={min_cluster_size}, cluster_selection_method={cluster_selection_method}, metric={hdbscan_metric})")

    # Tạo UMAP và HDBSCAN
    umap_model = UMAP(n_neighbors=n_neighbors, n_components=n_components, min_dist=min_dist, metric=umap_metric, random_state=42)
    hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, cluster_selection_method=cluster_selection_method, metric=hdbscan_metric, prediction_data=True, gen_min_span_tree=True)

    # BERTopic
    topic_model = BERTopic(
        # embedding_model=lambda docs: phobert_embedder(docs, show_progress_bar=False),
        embedding_model=lambda docs: embeddings(docs, show_progress_bar=False),
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer_model,
        # ctfidf_model=ctfidf_model,
        representation_model=representation_model,
        language="multilingual",
        calculate_probabilities=True,
        verbose=False
    )

    # Train
    topics, _ = topic_model.fit_transform(df['text'].tolist())
    # topics, _ = topic_model.fit_transform(list_of_docs, embeddings)

    # Tính DBCV score
    dbcv_score = topic_model.hdbscan_model.relative_validity_

    # Tính % outliers
    outlier_ratio = topics.count(-1) / len(topics)

    print(f"📊 DBCV: {round(dbcv_score, 3)} | Outlier Ratio: {round(outlier_ratio * 100, 2)}%")

    # Kết hợp: Ưu tiên DBCV cao nhất, sau đó là outlier thấp
    composite_score = dbcv_score - outlier_ratio  # hoặc dùng trọng số: alpha*DBCV - beta*outlier

    if composite_score > best_score:
        best_score = composite_score
        best_model = topic_model
        best_params = (n_neighbors, n_components, min_dist, umap_metric, min_cluster_size, cluster_selection_method, hdbscan_metric)

print(best_params)

topics, probs = best_model.fit_transform(df['text'].tolist())
# topics, probs = best_model.fit_transform(list_of_docs, embeddings)


🔍 Grid 1/36: UMAP(n_neighbors=4, min_dist=0.0, metric=cosine), HDBSCAN(min_cluster_size=4, cluster_selection_method=eom, metric=euclidean)
📊 DBCV: 0.641 | Outlier Ratio: 3.29%

🔍 Grid 2/36: UMAP(n_neighbors=4, min_dist=0.0, metric=cosine), HDBSCAN(min_cluster_size=4, cluster_selection_method=leaf, metric=euclidean)
📊 DBCV: 0.641 | Outlier Ratio: 3.29%

🔍 Grid 3/36: UMAP(n_neighbors=4, min_dist=0.0, metric=cosine), HDBSCAN(min_cluster_size=6, cluster_selection_method=eom, metric=euclidean)
📊 DBCV: 0.424 | Outlier Ratio: 14.86%

🔍 Grid 4/36: UMAP(n_neighbors=4, min_dist=0.0, metric=cosine), HDBSCAN(min_cluster_size=6, cluster_selection_method=leaf, metric=euclidean)
📊 DBCV: 0.434 | Outlier Ratio: 14.96%

🔍 Grid 5/36: UMAP(n_neighbors=4, min_dist=0.0, metric=cosine), HDBSCAN(min_cluster_size=8, cluster_selection_method=eom, metric=euclidean)
📊 DBCV: 0.395 | Outlier Ratio: 19.43%

🔍 Grid 6/36: UMAP(n_neighbors=4, min_dist=0.0, metric=cosine), HDBSCAN(min_cluster_size=8, cluster_selection_

In [34]:
# Tính DBCV score
dbcv_score = best_model.hdbscan_model.relative_validity_

# Tính % outliers
outlier_ratio = topics.count(-1) / len(topics)

print(f"📊 DBCV: {round(dbcv_score, 3)} | Outlier Ratio: {round(outlier_ratio * 100, 2)}%")

📊 DBCV: 0.641 | Outlier Ratio: 3.29%


In [32]:
df['topic'] = topics
df

,id,event_id,text,topic
0,0,1,Các nước chia buồn vụ tai nạn máy bay của Ai C...,23
1,1,1,Máy bay Ai Cập rơi: Những câu hỏi cho chính ph...,0
2,2,1,Giải mã bí ẩn máy bay rơi của EgyptAir . Phó C...,74
3,3,1,Phát hiện mảnh vỡ nghi của máy bay MS804 gặp n...,-1
4,4,1,Máy bay EgyptAir có thể bị tấn công bằng tên l...,42
...,...,...,...,...
1940,1940,300,Xử lý xong sự cố ở sân bay Buôn Ma Thuột . Chi...,6
1941,1941,300,Đã khắc phục xong sự cố đường băng sân bay Buô...,6
1942,1942,300,Khắc phục xong sự cố Cảng hàng không Buôn Ma T...,6
1943,1943,300,Khắc phục xong sự cố tại Cảng Hàng không Buôn ...,6


In [33]:
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.preprocessing import LabelEncoder

df_eval = df[df["topic"] != -1].copy()
ari = adjusted_rand_score(df_eval["event_id"], df_eval["topic"])
nmi = normalized_mutual_info_score(df_eval["event_id"], df_eval["topic"])

print(f"ARI: {ari:.4f}")
print(f"NMI: {nmi:.4f}")

ARI: 0.7968
NMI: 0.9597


In [ ]:
topic_to_docs = defaultdict(list)

for doc, topic in zip(list_of_docs, topics):
    topic_to_docs[topic].append(doc)

df = pd.DataFrame([
    {"Topic": topic, "Docs": topic_docs, "Count": len(topic_docs)}
    for topic, topic_docs in topic_to_docs.items()
])

result = []
for _, row in df.iterrows():
    result.append({
        "topic": row["Topic"],
        "count": row["Count"],
        "docs": row["Docs"]
    })

In [20]:
# JSON
with open(fr"C:\Users\rreip\Downloads\HK6\IE403\Đồ án\Event_Cluster.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)